In [1]:
import datetime

In [3]:
MODEL_NAME = "premier_league"
MODEL_VERSION = "v1.0"
MODEL_TYPE = "win_outcome"

In [4]:
# UPDATING THE GITHUB OF THE DATASETS
MODEL_TIME = datetime.datetime.now().strftime("%d-%M-%Y_%T")
!git add -A
!git commit -m "updates: {MODEL_NAME}-{MODEL_VERSION}-{MODEL_TYPE}: bot commit - {MODEL_TIME}"
!git push

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date
